In [38]:
import numpy as np
import random
import pandas as pd
import os, glob
import scipy.stats as stats

import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.metrics import plot_confusion_matrix, classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
f_path = './[배포용]+MINIST+Data/'
files = [os.path.join(f_path, f) for f in os.listdir(f_path)]

In [3]:
data_set_y = []
data_set_x = []

for f in files:
    y = int(f.split('/')[-1].split('_')[0])
    x = pd.read_csv(f, header=None).values
    data_set_x.append(x)
    data_set_y.append(y)

df_y = pd.DataFrame(data_set_y, columns=['label'])
data_set_x = np.array(data_set_x)


In [167]:

def polynomial_basis(x, degree):
    """
    x: 입력 데이터
    degree: 다항식 차수
    """
    x = np.array(x)

    # 다항식 차수에 따라 다항식 생성
    polynomial = np.ones((x.shape[0], 1))
    for i in range(1, degree+1):
        polynomial = np.concatenate([polynomial, x**i], axis=1)

    return polynomial



def feature_and_basis(temp_image,verbose=False):
    random.seed(10)
    sel_feature_num = 5

    ## num basis ## 
    num_basis = 5
    #### 특징을 제시해주신 10가지를 모두 생성 후 random 하게 5개를 선택하는 방식입니다. ####

    ###################### 특징 1, 2 생성 ######################
    ## 가로축 projection 후 기대 값, 분산 계산 ##
    projection_v = np.sum(temp_image, axis=1)
    pdf_v = projection_v / np.sum(projection_v, keepdims=True)
    expected_v = np.sum(projection_v * pdf_v, keepdims=True)
    variance_v = np.sum((projection_v - expected_v)**2 * pdf_v)
    expected_v = expected_v.reshape(-1)

    ###################### 특징 3, 4 생성 ######################
    ## 세로축 projection 후 기대 값, 분산 계산 ##

    projection_h = np.sum(temp_image, axis=0)
    pdf_h = projection_h / np.sum(projection_h, keepdims=True)
    expected_h = np.sum(projection_h * pdf_h,  keepdims=True)
    variance_h = np.sum((projection_h - expected_h)**2 * pdf_h)
    expected_h = expected_h.reshape(-1)


    ###################### 특징 5 ~ 10 생성 ######################

    ##  대각선 projection 후 기대값, 분산, 0의 개수 계산 ##
    ##  좌우 반전 후 대각선 projection 후 기대값, 분산, 0의 개수 계산 ##

    diag = np.diag(temp_image)
    pdf_d = diag / np.sum(diag)
    expected_d = np.sum(diag * pdf_d)
    variance_d = np.sum((diag - expected_d)**2 *pdf_d)
    num_zeros = np.count_nonzero(diag==0)

    a_diag = np.diag(np.fliplr(temp_image))
    pdf_a_d = a_diag / np.sum(a_diag)
    expected_a_d = np.sum(a_diag * pdf_a_d)
    variance_a_d = np.sum((a_diag - expected_a_d)**2 *pdf_a_d)
    num_zeros_a = np.count_nonzero(a_diag == 0)

    features = np.vstack([expected_v, variance_v, expected_h, variance_h, 
                          expected_d, variance_d, num_zeros, expected_a_d, variance_a_d, num_zeros_a])
    feature_label = np.array(['기대값_가로축', '분산_가로축', '기대값_세로축', '분산_세로축', '기대값_diagonal', '분산_diagonal', 'zero_cnt_diagonal',
                    '기대값_Anti-diagonal', '분산_Anti-diagonal', 'zero_cnt_Anti-diagonal'])
    ## random select ##
    feat_idx = random.sample(list(range(0,10)), sel_feature_num)
    features = features[feat_idx,:].T

    func = 'poly' # or 'gaussian'

    #### polynomial basis function ####
    feature_out = polynomial_basis(x=features, degree=num_basis)
    
    basis_fea_set = np.resize(feature_out,(1, sel_feature_num*num_basis+1))
    if verbose:
        print('Selected Features :\n', feature_label[feat_idx])
        print('특징 수 :',sel_feature_num,',', '기저 함수 수 :', num_basis)
        print(basis_fea_set.shape)

    return basis_fea_set


In [168]:
all_data_x = np.array([feature_and_basis(data).reshape(-1) for data in data_set_x])

all_data_x.shape

(1500, 26)

In [169]:

# data_split : 전체 data 분할 및 label 설정하는 함수
def data_split(input_x, input_y):
    """ 
    """

    train_test_dict = dict()
    data_x = input_x
    y = input_y.values if isinstance(input_y, pd.DataFrame) else input_y

    X_train, X_test, y_train, y_test = train_test_split(data_x, y, test_size=0.2, random_state=2020,shuffle=True)

    # scaler = StandardScaler()
    # scaler = scaler.fit(X_train)

    # X_train = scaler.transform(X_train)
    # X_test = scaler.transform(X_test)

    train_test_dict['test_x'] = X_test
    train_test_dict['test_y'] = y_test
    train_test_dict['train_x'] = X_train
    train_test_dict['train_y'] = y_train

    return train_test_dict

train_test_dict = data_split(input_x=all_data_x, input_y=df_y)

In [170]:
train_test_dict['train_y'].shape, train_test_dict['train_x'].shape

((1200, 1), (1200, 26))

In [171]:
model = LinearRegression()
model.fit(train_test_dict['train_x'], train_test_dict['train_y'])

LinearRegression()

In [172]:
y_pred = model.predict(train_test_dict['test_x'])


y_pred[y_pred < 0.5] = 0
y_pred[(0.5<=y_pred) & (y_pred < 1.5)] = 1
y_pred[1.5<=y_pred] = 2


In [173]:
print('acc',accuracy_score(y_true=train_test_dict['test_y'], y_pred=y_pred))

print(classification_report(y_true=train_test_dict['test_y'], y_pred=y_pred))
confusion_matrix(y_true=train_test_dict['test_y'], y_pred=y_pred)

acc 0.6066666666666667
              precision    recall  f1-score   support

           0       0.90      0.47      0.62        99
           1       0.49      0.95      0.65       107
           2       0.80      0.35      0.49        94

    accuracy                           0.61       300
   macro avg       0.73      0.59      0.59       300
weighted avg       0.73      0.61      0.59       300



array([[ 47,  46,   6],
       [  3, 102,   2],
       [  2,  59,  33]], dtype=int64)

In [174]:
weight = pd.DataFrame(model.coef_.T)
weight

0
0   4.581382e-02
1  -1.647630e+01
2   1.742416e+00
3  -5.157478e+01
4   1.265380e-01
5   1.908859e+03
6   1.992803e+00
7  -5.696815e-01
8   6.005497e+00
9  -9.589245e-03
10 -4.913468e+03
11 -1.146510e-01
12  8.532081e-02
13 -3.387716e-01
14  5.091419e-04
15  6.268262e+03
16  3.143092e-03
17 -5.828222e-03
18  9.263862e-03
19 -1.482414e-05
20 -3.980264e+03
21 -3.300958e-05
22  1.458578e-04
23 -9.845508e-05
24  1.612374e-07
25  1.008861e+03

In [175]:
weight.to_csv('학번_weight.csv',index=False)

In [95]:
## 검증 코드 검증##
x_test = np.array([], dtype='float32')
x_test = np.resize(x_test, (0,16))
print(x_test.shape)
for d in data_set_x:
    print(d.shape)
    temp=feature_and_basis(d)
    x_test = np.concatenate((x_test, temp), axis=0)

x_test.shape

(0, 16)
(28, 28)
Selected Features :
 ['zero_cnt_Anti-diagonal' '기대값_가로축' 'zero_cnt_diagonal' '분산_세로축'
 '기대값_diagonal']
특징 수 : 5 , 기저 함수 수 : 3
(1, 16)
(28, 28)
Selected Features :
 ['zero_cnt_Anti-diagonal' '기대값_가로축' 'zero_cnt_diagonal' '분산_세로축'
 '기대값_diagonal']
특징 수 : 5 , 기저 함수 수 : 3
(1, 16)
(28, 28)
Selected Features :
 ['zero_cnt_Anti-diagonal' '기대값_가로축' 'zero_cnt_diagonal' '분산_세로축'
 '기대값_diagonal']
특징 수 : 5 , 기저 함수 수 : 3
(1, 16)
(28, 28)
Selected Features :
 ['zero_cnt_Anti-diagonal' '기대값_가로축' 'zero_cnt_diagonal' '분산_세로축'
 '기대값_diagonal']
특징 수 : 5 , 기저 함수 수 : 3
(1, 16)
(28, 28)
Selected Features :
 ['zero_cnt_Anti-diagonal' '기대값_가로축' 'zero_cnt_diagonal' '분산_세로축'
 '기대값_diagonal']
특징 수 : 5 , 기저 함수 수 : 3
(1, 16)
(28, 28)
Selected Features :
 ['zero_cnt_Anti-diagonal' '기대값_가로축' 'zero_cnt_diagonal' '분산_세로축'
 '기대값_diagonal']
특징 수 : 5 , 기저 함수 수 : 3
(1, 16)
(28, 28)
Selected Features :
 ['zero_cnt_Anti-diagonal' '기대값_가로축' 'zero_cnt_diagonal' '분산_세로축'
 '기대값_diagonal']
특징 수 : 5 , 기저 함수 수 : 3

(1500, 16)

In [82]:
data_x = feature_and_basis(temp_image=data_set_x[0])

Selected Features :
 ['분산_가로축' '기대값_세로축' '분산_diagonal' 'zero_cnt_Anti-diagonal'
 'zero_cnt_diagonal']
특징 수 : 5 , 기저 함수 수 : 3
(1, 16)


((1200, 1), (1200, 16))